使用预训练BERT做新闻文本分类，轻松达到分数0.88以上-Tianchi: Lab
https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.36.6406b751tyh47J&postId=377880

In [1]:
!pip install torch pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 5.1 MB/s 
     |████████████████████████████████| 419 kB 53.6 MB/s 
     |████████████████████████████████| 140 kB 59.8 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 59.1 MB/s 
     |████████████████████████████████| 144 kB 63.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.1 MB/s 
     |████████████████████████████████| 101 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.1 MB/s 


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
# from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.functional import f1_score

from transformers import AutoTokenizer, BertForSequenceClassification

In [4]:
target = torch.tensor([0., 1., 2., 0., 1., 2.])
preds = torch.tensor([0., 2., 1., 0., 0., 1.])
target = target.type(torch.int32)
preds = preds.type(torch.int32)
f1_score(preds, target, num_classes=3)
# preds, target

tensor(0.3333)

In [5]:
class CustomDataset(Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.comment_text = self.df.text
        self.max_len = max_len
        if 'label' in self.df:
            self.targets = self.df.label

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = self.comment_text[index]

        encode_dict = self.tokenizer.encode_plus(
            comment_text,
            add_special_tokens=True,    # 添加特殊token [CLS], [SEP]
            truncation=True,    # 是否截断超长句子
            max_length=self.max_len,    # 句子最大长度
            padding='max_length',   # 长度不够的填充至最大长度
            return_attention_mask=True,  # 是否返回attention mask
            return_tensors='pt', # 返回tensor类型
        )
        ids = encode_dict['input_ids']
        mask = encode_dict['attention_mask']

        ids = ids.squeeze(0)    # (1, max_len) -> (max_len)
        mask = mask.squeeze(0)  # (1, max_len) -> (max_len)
        
        if 'label' in self.df:
            target = self.targets[index]
            return {"ids" : ids, "mask": mask, "target": target}
        return {"ids": ids, "mask": mask}

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
# read the csv file
df = pd.read_csv('/content/drive/MyDrive/data/sdp2022/task1_train_dataset.csv')
  
# display dataset
print(df)

       index                                              title  \
0          0  Activation of nitrofurazone by azoreductases: ...   
1          1  Organisation and delivery of home care re-able...   
2          2  Relationships between anthocyanins and other c...   
3          3  New Insights into the Mechanisms of Water-Stre...   
4          4  Extensive chordate and annelid macrosynteny re...   
...      ...                                                ...   
51555  51555  BALB/c mice deficient in CD4(+) T cell IL-4R a...   
51556  51556         The Afterlife of Parliamentary Sovereignty   
51557  51557  Metabolomic Proﬁling of In Vivo Plasma Respons...   
51558  51558  Constant flux relation for diffusion-limited c...   
51559  51559  Sequence determinants for the tandem recogniti...   

            first_author                                        description  \
0            M Nakanishi                                                NaN   
1                  Allen             

In [8]:
df = df[['title','theme']]
df

,title,theme
0,Activation of nitrofurazone by azoreductases: ...,"Allied Health Professions, Dentistry, Nursing ..."
1,Organisation and delivery of home care re-able...,Social Work and Social Policy
2,Relationships between anthocyanins and other c...,"Agriculture, Veterinary and Food Science"
3,New Insights into the Mechanisms of Water-Stre...,Earth Systems and Environmental Sciences
4,Extensive chordate and annelid macrosynteny re...,Biological Sciences
...,...,...
51555,BALB/c mice deficient in CD4(+) T cell IL-4R a...,"Allied Health Professions, Dentistry, Nursing ..."
51556,The Afterlife of Parliamentary Sovereignty,Law
51557,Metabolomic Proﬁling of In Vivo Plasma Respons...,"Agriculture, Veterinary and Food Science"
51558,Constant flux relation for diffusion-limited c...,Mathematical Sciences


In [9]:
df["theme"] = df["theme"].astype('category')
df.dtypes

df["theme_id"] = df["theme"].cat.codes
df.tail()

,title,theme,theme_id
51555,BALB/c mice deficient in CD4(+) T cell IL-4R a...,"Allied Health Professions, Dentistry, Nursing ...",2
51556,The Afterlife of Parliamentary Sovereignty,Law,23
51557,Metabolomic Proﬁling of In Vivo Plasma Respons...,"Agriculture, Veterinary and Food Science",1
51558,Constant flux relation for diffusion-limited c...,Mathematical Sciences,24
51559,Sequence determinants for the tandem recogniti...,"Allied Health Professions, Dentistry, Nursing ...",2


In [10]:
df = df[['title','theme_id']]

In [ ]:
df.columns = ['text', 'label']


In [2]:
df.columns = ['text','label']
df

NameError: ignored

In [1]:
 df.label

NameError: ignored

In [14]:
max_len = 256
train_dataset = CustomDataset(df, tokenizer, max_len=max_len)
# test_dataset = CustomDataset(test_set, tokenizer, max_len=max_len)

In [23]:
class Classifier(pl.LightningModule):
    def __init__(self, hparams):
        super(Classifier, self).__init__()
        
        # 不返回attention weights和hidden states
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=36, output_attentions=False, output_hidden_states=False)
        
        self.batch_size = hparams["batch_size"]
        self.lr = hparams["lr"]
        self.gamma = hparams["gamma"]

    def forward(self, batch):
        if len(batch) == 3:
            input_ids, attention_masks, labels = batch['ids'], batch['mask'], batch['target']
            loss, logits = self.model(input_ids, attention_mask=attention_masks, labels=labels, token_type_ids=None, return_dict=False)
            return loss, logits
        else:
            input_ids, attention_mask = batch['ids'], batch['mask']
            logits = self.model(input_ids, token_type_ids=None, attention_mask=attention_mask)
            return logits[0]

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, self.gamma)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        loss, logits = self.forward(batch)
        
        pred_flat = torch.argmax(logits, dim=1).flatten().unsqueeze(1)
        
        labels_flat = batch['target'].flatten()
        # acc = torch.sum(pred_flat == labels_flat) / len(labels_flat)
        # 计算f1 score
        f1 = f1_score(pred_flat, labels_flat, average="macro", num_classes=36)
        self.log("loss", loss)
        self.log("train_f1", f1)
        return loss

    # def training_epoch_end(self, outputs):
    #     avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
    #     avg_f1 = torch.stack([x["train_f1"] for x in outputs]).mean()
    #     self.log("step", self.trainer.current_epoch)
    #     self.log("avg_loss", avg_loss)
    #     self.log("avg_acc", avg_f1)

    # def validation_step(self, batch, batch_idx):
    #     loss, logits = self.forward(batch)
    #     pred_flat = torch.argmax(logits, dim=1).flatten()
    #     labels_flat = batch[2].flatten()
    #     f1 = f1_score(pred_flat, labels_flat, average="macro")
    #     return {"val_f1": f1}

    # def validation_epoch_end(self, outputs):
    #     avg_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
    #     self.log("val_f1", avg_f1)
    
    def train_dataloader(self):
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=32)
        return train_loader

    # def val_dataloader(self):
    #     valid_loader = DataLoader(valid_dataset, batch_size=self.batch_size, sampler=RandomSampler(valid_dataset))
    #     return valid_loader

    def test_dataloader(self):
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=32)
        return test_loader

In [ ]:
hparams = {
    "batch_size": 8,
    "lr": 3e-5,
    "gamma": 0.8,
}

model = Classifier(hparams)
checkpoint_callback = ModelCheckpoint(
    monitor="train_f1",
    dirpath="./",
    mode='max',
    filename='best',
)
trainer = pl.Trainer(max_epochs=3,log_every_n_steps=1000, callbacks=[checkpoint_callback])
trainer.fit(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training: 0it [00:00, ?it/s]